## Mapping the Data
In `02_st-cleaing-5yr.ipynb` we found that there are some hotspots for street cleaning violations. Here let's map them out.  
Alternatively, I also marked hotposts on [Google MyMap](https://www.google.com/maps/d/edit?mid=1hqpDiDN_2dWwhyBaKwjsPb2qxoinSww&usp=sharing) for better view with analysis neighborhood.

### Import Data

In [1]:
import pandas as pd
import requests

In [2]:
# Geocoding addresses from Pandas
# Import citation_st_cleaning_5yr.csv
st_cleaning_map = pd.read_csv('citation_st_cleaning_5yr.csv',
                              dtype={'Citation Number': object} ,
                              parse_dates=['Citation Issued DateTime'])

In [3]:
st_cleaning_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2376891 entries, 0 to 2376890
Data columns (total 15 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   Citation Number               object        
 1   Citation Issued DateTime      datetime64[ns]
 2   Violation                     object        
 3   Violation Description         object        
 4   Citation Location             object        
 5   Vehicle Plate State           object        
 6   Vehicle Plate                 object        
 7   Fine Amount                   float64       
 8   Date Added                    object        
 9   geom                          object        
 10  Neighborhoods                 float64       
 11  SF Find Neighborhoods         float64       
 12  Current Police Districts      float64       
 13  Current Supervisor Districts  float64       
 14  Analysis Neighborhoods        float64       
dtypes: datetime64[ns](1), float64(6)

In [4]:
# Add new column with complete address for 'Citation Location' for geocoding
st_cleaning_map['Location_new'] = st_cleaning_map['Citation Location'].astype(str) + ', San Francisco, USA'

In [5]:
st_cleaning_map.head(10)

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Location_new
0,949367440,2022-03-23 02:33:00,TRC7.2.22,STR CLEAN,720 TURK ST,CA,8U09234,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"720 TURK ST, San Francisco, USA"
1,949375335,2022-03-24 09:30:00,TRC7.2.22,STR CLEAN,19 RIVAS AVE,CA,8GOA162,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"19 RIVAS AVE, San Francisco, USA"
2,949395613,2022-03-21 13:05:00,TRC7.2.22,STR CLEAN,1405 BAKER ST,CA,5JZT981,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"1405 BAKER ST, San Francisco, USA"
3,949411481,2022-03-29 00:15:00,TRC7.2.22,STR CLEAN,420 NATOMA ST,CA,8DEJ983,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"420 NATOMA ST, San Francisco, USA"
4,949412855,2022-03-22 12:25:00,TRC7.2.22,STR CLEAN,717 WALLER ST,CA,6GOF707,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"717 WALLER ST, San Francisco, USA"
5,894375451,2018-02-20 10:15:00,TRC7.2.22,STR CLEAN,3898 WASHINGTON ST,CA,ELBA227,73.0,08/10/2021 12:00:00 AM,POINT (-122.45708101499997 37.788961003000054),10.0,10.0,8.0,6.0,31.0,"3898 WASHINGTON ST, San Francisco, USA"
6,895929086,2018-03-29 09:35:00,TRC7.2.22,STR CLEAN,812 SILLIMAN ST,CA,7TGY978,73.0,08/10/2021 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"812 SILLIMAN ST, San Francisco, USA"
7,938910103,2021-05-03 12:13:00,TRC7.2.22,STR CLEAN,1820 25TH ST,CA,6HKN775,83.0,08/08/2021 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"1820 25TH ST, San Francisco, USA"
8,900852466,2018-08-13 08:24:00,TRC7.2.22,STR CLEAN,173 ARKANSAS ST,CA,56907K2,76.0,02/13/2022 12:00:00 AM,POINT (-122.39854476899995 37.76402533000004),54.0,54.0,1.0,9.0,26.0,"173 ARKANSAS ST, San Francisco, USA"
9,900915735,2018-08-17 09:40:00,TRC7.2.22,STR CLEAN,224 GARFIELD ST,CA,7T62119,76.0,02/13/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"224 GARFIELD ST, San Francisco, USA"


### Clean Location Data

In [63]:
# There are locations with NaN but have geometry location data.
st_cleaning_map[
    (st_cleaning_map['geom'].notnull()) &
    (st_cleaning_map['Citation Location'].isna())
].sort_values(['Citation Issued DateTime'])

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Location_new
1194049,920109820,2018-06-13 06:55:00,TRC7.2.22,STR CLEAN,NaN,CA,655TTU,0.0,06/16/2018 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1195948,920109831,2018-06-13 07:04:00,TRC7.2.22,STR CLEAN,NaN,CA,5YYG,0.0,06/16/2018 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1201292,920112955,2018-06-20 02:22:00,TRC7.2.22,STR CLEAN,NaN,CA,8ARF698,0.0,06/23/2018 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1199971,920112970,2018-06-20 02:50:00,TRC7.2.22,STR CLEAN,NaN,CA,7XUX544,0.0,06/23/2018 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1201897,920113611,2018-06-20 03:13:00,TRC7.2.22,STR CLEAN,NaN,CA,1483242,0.0,06/23/2018 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492632,1005224194,2019-10-03 00:00:00,TRC7.2.22,STR CLEAN,NaN,CA,NaN,79.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1659480,1005938765,2019-11-12 12:05:00,TRC7.2.22,STR CLEAN,NaN,CA,6WJP697,79.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
632427,926241094,2019-12-17 00:00:00,TRC7.2.22,STR CLEAN,NaN,CA,6MLP864,79.0,11/23/2021 12:00:00 AM,POINT (-122.43454601699995 37.76714601000003),28.0,28.0,7.0,5.0,5.0,"nan, San Francisco, USA"
1318578,1005860903,2020-01-14 06:37:00,TRC7.2.22,STR CLEAN,NaN,CA,3WLS254,79.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"


In [69]:
# Out of 111 rows of data, there are 100 rows that the 'Fine Amount' is $0.0 so only geocode the rest 11 rows data.
# ???
st_cleaning_map[
    (st_cleaning_map['geom'].notnull()) &
    (st_cleaning_map['Citation Location'].isna()) &
    (st_cleaning_map['Fine Amount'] == 0.0)
].sort_values(['Citation Issued DateTime'])

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Location_new
1194049,920109820,2018-06-13 06:55:00,TRC7.2.22,STR CLEAN,NaN,CA,655TTU,0.0,06/16/2018 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1195948,920109831,2018-06-13 07:04:00,TRC7.2.22,STR CLEAN,NaN,CA,5YYG,0.0,06/16/2018 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1201292,920112955,2018-06-20 02:22:00,TRC7.2.22,STR CLEAN,NaN,CA,8ARF698,0.0,06/23/2018 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1199971,920112970,2018-06-20 02:50:00,TRC7.2.22,STR CLEAN,NaN,CA,7XUX544,0.0,06/23/2018 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1201897,920113611,2018-06-20 03:13:00,TRC7.2.22,STR CLEAN,NaN,CA,1483242,0.0,06/23/2018 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697267,920637351,2019-06-23 07:19:00,TRC7.2.22,STR CLEAN,NaN,CA,6FHZ184,0.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1453950,920637900,2019-06-23 07:23:00,TRC7.2.22,STR CLEAN,NaN,DD,20191B4,0.0,08/05/2019 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1772251,920778736,2019-07-05 13:02:00,TRC7.2.22,STR CLEAN,NaN,DD,20191B6,0.0,08/05/2019 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1780605,920780711,2019-07-05 13:41:00,TRC7.2.22,STR CLEAN,NaN,CA,8CVU107,0.0,08/05/2019 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"


In [70]:
st_cleaning_map[
    (st_cleaning_map['geom'].notnull()) &
    (st_cleaning_map['Citation Location'].isna()) &
    (st_cleaning_map['Fine Amount'] != 0.0)
].sort_values(['Citation Issued DateTime'])

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Location_new
1730867,1005474271,2018-08-30 07:21:00,TRC7.2.22,STR CLEAN,NaN,CA,6TZX367,76.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1375057,1005195365,2019-02-13 12:06:00,TRC7.2.22,STR CLEAN,NaN,CA,7AJE508,76.0,04/11/2021 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1568823,1005518850,2019-04-15 21:47:00,TRC7.2.22,STR CLEAN,NaN,NaN,NaN,76.0,12/04/2019 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
2164272,1005015281,2019-08-02 09:13:00,TRC7.2.22,STR CLEAN,NaN,TX,LVK7849,79.0,07/24/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
2161961,1005773801,2019-08-02 12:15:00,TRC7.2.22,STR CLEAN,NaN,CA,6DQT925,79.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1333005,1005771734,2019-08-08 13:14:00,TRC7.2.22,STR CLEAN,NaN,CA,8HED311,79.0,02/21/2022 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1824495,1005819054,2019-09-12 07:28:00,TRC7.2.22,STR CLEAN,NaN,CA,8DKL101,79.0,10/15/2021 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1492632,1005224194,2019-10-03 00:00:00,TRC7.2.22,STR CLEAN,NaN,CA,NaN,79.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
1659480,1005938765,2019-11-12 12:05:00,TRC7.2.22,STR CLEAN,NaN,CA,6WJP697,79.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,"nan, San Francisco, USA"
632427,926241094,2019-12-17 00:00:00,TRC7.2.22,STR CLEAN,NaN,CA,6MLP864,79.0,11/23/2021 12:00:00 AM,POINT (-122.43454601699995 37.76714601000003),28.0,28.0,7.0,5.0,5.0,"nan, San Francisco, USA"


In [66]:
# Save data that needs to geocode to another dataframe. Save as .csv in case as well.
st_cleaning_geocode = st_cleaning_map[
    (st_cleaning_map['geom'].notnull()) &
    (st_cleaning_map['Citation Location'].isna())
].sort_values(['Citation Issued DateTime']).reset_index()

In [67]:
# Export dataframe as .csv
st_cleaning_geocode.to_csv('st_cleaning_geocode.csv', index=False)

In [64]:
# There are 5 rows of data that have no location data at all
st_cleaning_map[
    (st_cleaning_map['geom'].isna()) &
    (st_cleaning_map['Citation Location'].isna())
].sort_values(['Citation Issued DateTime'])

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Location_new
474695,1005977910,2020-09-04 07:20:00,TRC7.2.22,STR CLEAN,NaN,CA,6LZC897,79.0,10/17/2021 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"nan, San Francisco, USA"
545810,1006049240,2020-09-09 10:10:00,TRC7.2.22,STR CLEAN,NaN,CA,5DTY745,79.0,11/13/2020 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"nan, San Francisco, USA"
191566,1006046602,2021-02-17 10:14:00,TRC7.2.22,STR CLEAN,NaN,CA,KNGJSRI,83.0,04/21/2021 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"nan, San Francisco, USA"
1138236,952359984,2022-06-21 02:46:00,TRC7.2.22,STR CLEAN,NaN,DD,20221B2,0.0,06/26/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"nan, San Francisco, USA"
1159826,952578900,2022-07-02 00:12:00,TRC7.2.22,STR CLEAN,NaN,DD,20221B3,0.0,07/07/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,"nan, San Francisco, USA"


### Where are the citation hotspots in recent five years?
From last notebook, we know that top 10 hotspots are all close to each location: 5 locations on the Mission St., 3 on Taylor St., and 2 on Jones St.  
If we put the locations on the map, 5 of the locations are in the Tenderloin district, 4 in SOMA district, and 1 in Financial district.

In [6]:
# Top 10 hotspots
st_cleaning_map['Citation Location'].value_counts().nlargest(10)

1190 MISSION ST    459
111 TAYLOR ST      339
540 JONES ST       327
1188 MISSION ST    287
814 MISSION ST     282
555 MISSION ST     270
556 JONES ST       262
520 TAYLOR ST      251
1018 MISSION ST    249
501 TAYLOR ST      242
Name: Citation Location, dtype: int64

In [8]:
# Top 50 hotspots
st_cleaning_map['Citation Location'].value_counts().nlargest(50)

1190 MISSION ST              459
111 TAYLOR ST                339
540 JONES ST                 327
1188 MISSION ST              287
814 MISSION ST               282
555 MISSION ST               270
556 JONES ST                 262
520 TAYLOR ST                251
1018 MISSION ST              249
501 TAYLOR ST                242
50 CLEARY CT                 239
1000 GREAT HWY               235
3500 19TH ST                 234
1190 MISSION STREET          226
3048 16TH ST                 219
555 ELLIS ST                 216
450 JONES ST                 215
480 ELLIS ST                 202
520 SOUTH VAN NESS AVE       198
355 BERRY ST                 197
420 EDDY ST                  193
230 GOLDEN GATE AVE          189
1825 MISSION ST              189
1036 MISSION ST              188
350 PARNASSUS AVE            186
845 HYDE ST                  185
225 TAYLOR ST                184
420 JONES ST                 184
575 MISSION ST               183
65 CLEARY CT                 182
1026 MISSI

In [9]:
# Top 100 hotspots
st_cleaning_map['Citation Location'].value_counts().nlargest(100)

1190 MISSION ST    459
111 TAYLOR ST      339
540 JONES ST       327
1188 MISSION ST    287
814 MISSION ST     282
                  ... 
1180 POST ST       138
30 DORE ST         138
101 POLK ST        138
222 TAYLOR ST      138
600 MINNA ST       137
Name: Citation Location, Length: 100, dtype: int64

### Hotspots each year?
Citation hotspots becomes more concentrated from 2018 to 2022.

#### 2018
Top 10 hotspots are located in: Golden Gate Park(3), SoMA(1), Financial District(1), Western Addition(4), Lakeshore(1).  
The top location for getting ticket is `1000 GREAT HWY` in Golden Gate Park Neighborhood that close to the beach.

In [40]:
# 2018 full list
map_count_2018 = st_cleaning_map[
    (st_cleaning_map['Citation Issued DateTime'] >= '2018-01-01') & 
    (st_cleaning_map['Citation Issued DateTime'] < '2019-01-01')
]
map_count_2018 = map_count_2018['Location_new'].value_counts()
map_count_2018

1000 GREAT HWY, San Francisco, USA       179
950 GREAT HWY, San Francisco, USA        142
1050 GREAT HWY, San Francisco, USA       138
814 MISSION ST, San Francisco, USA       121
555 MISSION ST, San Francisco, USA       104
                                        ... 
1752 LANE ST, San Francisco, USA           1
159 DAY ST, San Francisco, USA             1
206 BADEN ST, San Francisco, USA           1
328 TURK ST, San Francisco, USA            1
1933 O'FARRELL ST, San Francisco, USA      1
Name: Location_new, Length: 217787, dtype: int64

In [41]:
# 2018 top 10
map_count_2018.nlargest(10)

1000 GREAT HWY, San Francisco, USA    179
950 GREAT HWY, San Francisco, USA     142
1050 GREAT HWY, San Francisco, USA    138
814 MISSION ST, San Francisco, USA    121
555 MISSION ST, San Francisco, USA    104
50 CLEARY CT, San Francisco, USA       89
65 CLEARY CT, San Francisco, USA       82
60 CLEARY CT, San Francisco, USA       80
750 FONT BLVD, San Francisco, USA      76
40 CLEARY CT, San Francisco, USA       73
Name: Location_new, dtype: int64

#### 2019
Top 10 hotspots are located in: Western Addition, SoMA, Chinatown, Financial District, Potrero Hill, and Mission.  
All top 10 hotspots are located at the east side of the city.  
The top location for citation is `50 CLEARY CT` at Western Addition Neighborhood.

In [42]:
# 2019 full list
map_count_2019 = st_cleaning_map[
    (st_cleaning_map['Citation Issued DateTime'] >= '2019-01-01') & 
    (st_cleaning_map['Citation Issued DateTime'] < '2020-01-01')
]
map_count_2019 = map_count_2019['Location_new'].value_counts()
map_count_2019

50 CLEARY CT, San Francisco, USA           132
nan, San Francisco, USA                     97
555 MISSION ST, San Francisco, USA          89
400 FULTON ST, San Francisco, USA           80
814 MISSION ST, San Francisco, USA          76
                                          ... 
750 INDIANA ST, San Francisco, USA           1
7000 CALIFORNIA AVE, San Francisco, USA      1
195 07TH ST, San Francisco, USA              1
7495 GEARY BLVD, San Francisco, USA          1
1269 SILVER AVENUE, San Francisco, USA       1
Name: Location_new, Length: 247386, dtype: int64

In [43]:
# 2019 top 10
map_count_2019.nlargest(10)

50 CLEARY CT, San Francisco, USA           132
nan, San Francisco, USA                     97
555 MISSION ST, San Francisco, USA          89
400 FULTON ST, San Francisco, USA           80
814 MISSION ST, San Francisco, USA          76
1200 SAN BRUNO AVE, San Francisco, USA      72
318 HOWARD ST, San Francisco, USA           60
1009 MISSION ST, San Francisco, USA         58
450 RHODE ISLAND ST, San Francisco, USA     58
55 CLEARY CT, San Francisco, USA            58
Name: Location_new, dtype: int64

#### 2020

In [44]:
# 2020 full list
map_count_2020 = st_cleaning_map[
    (st_cleaning_map['Citation Issued DateTime'] >= '2020-01-01') & 
    (st_cleaning_map['Citation Issued DateTime'] < '2021-01-01')
]
map_count_2020 = map_count_2020['Location_new'].value_counts()
map_count_2020

590 MISSION ST, San Francisco, USA         74
110 08TH ST, San Francisco, USA            66
111 TAYLOR ST, San Francisco, USA          56
1190 MISSION ST, San Francisco, USA        51
1018 MISSION ST, San Francisco, USA        51
                                           ..
494 STANYAN ST, San Francisco, USA          1
3306 CLEMENT ST, San Francisco, USA         1
1643 JERROLD AVENUE, San Francisco, USA     1
185 27TH STREET, San Francisco, USA         1
55 EDGAR AVENUE, San Francisco, USA         1
Name: Location_new, Length: 224523, dtype: int64

In [45]:
# 2020 top 10
map_count_2020.nlargest(10)

590 MISSION ST, San Francisco, USA         74
110 08TH ST, San Francisco, USA            66
111 TAYLOR ST, San Francisco, USA          56
1190 MISSION ST, San Francisco, USA        51
1018 MISSION ST, San Francisco, USA        51
540 JONES ST, San Francisco, USA           50
1036 MISSION STREET, San Francisco, USA    49
10 12TH ST, San Francisco, USA             48
355 MCALLISTER ST, San Francisco, USA      46
1055 MASON ST, San Francisco, USA          44
Name: Location_new, dtype: int64

#### 2021

In [46]:
# 2021 full list
map_count_2021 = st_cleaning_map[
    (st_cleaning_map['Citation Issued DateTime'] >= '2021-01-01') & 
    (st_cleaning_map['Citation Issued DateTime'] < '2022-01-01')
]
map_count_2021 = map_count_2021['Location_new'].value_counts()
map_count_2021

1190 MISSION ST, San Francisco, USA        173
1190 MISSION STREET, San Francisco, USA    128
556 JONES ST, San Francisco, USA           127
1188 MISSION ST, San Francisco, USA        105
540 JONES ST, San Francisco, USA           104
                                          ... 
2961 BUSH STREET, San Francisco, USA         1
333 CRESTA VISTA DR, San Francisco, USA      1
25 FUENTE AVENUE, San Francisco, USA         1
346 SAN BENITO WAY, San Francisco, USA       1
7 SHOTWELL ST, San Francisco, USA            1
Name: Location_new, Length: 266479, dtype: int64

In [47]:
# 2021 top 10
map_count_2021.nlargest(10)

1190 MISSION ST, San Francisco, USA        173
1190 MISSION STREET, San Francisco, USA    128
556 JONES ST, San Francisco, USA           127
1188 MISSION ST, San Francisco, USA        105
540 JONES ST, San Francisco, USA           104
555 ELLIS ST, San Francisco, USA           103
101 POLK ST, San Francisco, USA             99
111 TAYLOR ST, San Francisco, USA           96
887 HOWARD STREET, San Francisco, USA       93
1188 MISSION STREET, San Francisco, USA     92
Name: Location_new, dtype: int64

#### 2022

In [48]:
# 2022 full list
map_count_2022 = st_cleaning_map[
    (st_cleaning_map['Citation Issued DateTime'] >= '2022-01-01')
]
map_count_2022 = map_count_2022['Location_new'].value_counts()
map_count_2022

1190 MISSION ST, San Francisco, USA    125
1018 MISSION ST, San Francisco, USA    101
540 JONES ST, San Francisco, USA       101
111 TAYLOR ST, San Francisco, USA       93
450 JONES ST, San Francisco, USA        87
                                      ... 
786 INDIANA ST, San Francisco, USA       1
2204 REVERE AVE, San Francisco, USA      1
417 TREAT AVE, San Francisco, USA        1
45 BOARDMAN PL, San Francisco, USA       1
2501 25TH AVE, San Francisco, USA        1
Name: Location_new, Length: 218714, dtype: int64

In [49]:
# 2022 top 10
map_count_2022.nlargest(10)

1190 MISSION ST, San Francisco, USA           125
1018 MISSION ST, San Francisco, USA           101
540 JONES ST, San Francisco, USA              101
111 TAYLOR ST, San Francisco, USA              93
450 JONES ST, San Francisco, USA               87
1188 MISSION ST, San Francisco, USA            82
50 JONES ST, San Francisco, USA                82
520 SOUTH VAN NESS AVE, San Francisco, USA     76
1333 JONES ST, San Francisco, USA              72
420 JONES ST, San Francisco, USA               71
Name: Location_new, dtype: int64